Auto-Clean-Test

In [4]:
import logging
import os
import shutil
import dill as pickle
import pandas as pd
import matplotlib.pyplot as plt
from streamline.runners.auto_runner import AutoRunner
from streamline.runners.clean_optimize_runner import OptimizeClean

In [5]:
with open("./data/DigenData/all_new_test_sets.pkl",'rb') as f:
    digennoise = pickle.load(f)

#print(digennoise.keys())

with open("./data/DigenData/all_new_test_sets_no_noise.pkl",'rb') as f:
    digen = pickle.load(f)

#print(digen.keys())
#print(digen['digen8_4426'])
Test= pd.DataFrame(digennoise['digen8_4426']['X'])
yval = pd.DataFrame(digennoise['digen8_4426']['y'])
current_noise = digennoise['digen8_4426']['noise']
yval.columns = ['y']
yval.rename(columns = {'0':'y'}, inplace = True)
out = pd.concat([Test, yval], axis=1)
out.index.name = 'InstanceId'
out.to_csv('./data/DigenStudy/digen8_4426.csv')
print(out)

                  X0        X1        X2        X3        X4        X5  \
InstanceId                                                               
0           1.764052  0.400157  0.978738  2.240893  1.867558 -0.977278   
1           0.144044  1.454274  0.761038  0.121675  0.443863  0.333674   
2          -2.552990  0.653619  0.864436 -0.742165  2.269755 -1.454366   
3           0.154947  0.378163 -0.887786 -1.980796 -0.347912  0.156349   
4          -1.048553 -1.420018 -1.706270  1.950775 -0.509652 -0.438074   
...              ...       ...       ...       ...       ...       ...   
9995       -0.030453  1.082681 -0.530580  0.602367 -2.138291 -0.056632   
9996        2.520830 -0.124884  0.018068 -0.998623 -1.262734 -0.565107   
9997       -0.545705  1.696766 -0.508186  0.003621 -1.018515  0.251562   
9998       -0.367787 -0.568358 -0.721248  0.198280 -0.217422  0.914757   
9999        0.428571 -0.164999  0.864963 -0.647662  0.720058 -0.337715   

                  X6        X7       

In [6]:
cleanrun = OptimizeClean(dataset_name=['digen1_6265'], n_trials=1, data_path='./data/testfolder')
cleanrun.run()
print(cleanrun.param)


  0%|          | 0/1 [00:00<?, ?it/s]INFO: ------------------------------------------------------- 
INFO: Loading Dataset: digen1_6265
INFO: Validating and Identifying Feature Types...
INFO: Running Initial EDA:
INFO: Initial Data Counts: ----------------
INFO: Instance Count = 10000
INFO: Feature Count = 10
INFO:     Categorical  = 0
INFO:     Quantitative = 10
INFO: Missing Count = 0
INFO:     Missing Percent = 0.0
INFO: Class Counts: ----------------
INFO: Class Count Information
INFO: 
   Class  Instances
0      1       5174
1      0       4826
INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
INFO: No non-binary categorical features, skipping categorical encoding
INFO: Top 10 Correlated Features
INFO: 
   Removed_Feature Correlated_Feature  Correlation
5               X0                 X5    -0.023813
15              X1              

categorical cut = 5
sig cutoff = 0.0198
feature missing = 0.7
cleaning miss = 0.6
correlation removal = 0.9
partition method: random
n_splits = 9

424 min runtime for 1 dataset with 100 trials. 
= 7 hours.

In [ ]:
# autotest = AutoRunner(dataset_names=['digen8_4426'], clean=False, class_label='y', instance_label='InstanceId')

In [ ]:
#output_csv = autotest.run()

In [ ]:
#print(output_csv)

#performance = pd.read_csv(output_csv)
#print(performance)
#print(performance['ROC AUC'].max())
#print(performance.loc[performance['ROC AUC'].idxmax()][0])